# Formulaciones qubo

Recordamos la formulacion lineal del problema de knapsack:


$$ max\{  p^{t}x\}$$
st
$$ w^{t}x \leq W$$

Las computadoras cuanticas no trabajan con formulaciones lineales, sino con formulaciones Q.U.B.O. "Quadratic Unconstrained Binary Optimization".

La idea, es encontrar una matriz $Q$ tal que el problema de hallar un $x$  que resuelva el problema de knapsack, sea analogo a resolver el problema:

$$ min\{  x^{t}Qx\}$$

De naturaleza cuadrática.

La matriz $Q$ se descompone de la siguiente forma:

$$Q = -Q_{objetivo}+\alpha Q_{peso}$$ 

con $\alpha \in \mathcal{R}$

La idea es que el problema queda: 

$$ min\{  x^{t}Qx\} = min\{  x^{t}(-Q_{objetivo}+\alpha Q_{peso})x\} = $$
$$ min\{  x^{t}(-Q_{objetivo})x + x^{t}(\alpha Q_{peso})x\}$$

Es decir, se busca una descomposición de dos matrices, tal que si se logra minimizar $x^{t}(-Q_{objetivo})x$ y $x^{t}(\alpha Q_{peso})x$ simultaneamente para el mismo $x$ entonces este $x$ será solución.

Podemos notar que hay un problema, minimizar una suma no es lo mismo que minimizar cada término por separado, de ahi el surgimiento de la constante $\alpha$ (a determinar) que garantice que la minimizacion sea coherente con el problema original

# Construccion de la matriz de funcion objetivo

Construiremos una matriz $Q_{objetivo}$ tal el problema $max\{  p^{t}x\}$ sea analogo a $min\{  x^{t}(-Q_{objetivo})x\}$

Esta matriz $Q$ se puede determinar de la siguiente manera:

 \begin{pmatrix}
    p_1 & 0 & \dots & 0 \\
    0 & p_2 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & p_n
  \end{pmatrix}

  Si hacemos las cuentas:

  $$  x^{t}(-Q_{objetivo})x = p_1x_1^2 + p_2x_2^2+ \dots + p_nx_n^2  $$
  y como $x_i \in \{0, 1\} \rightarrow x_i^2 = x_i$ 

  entonces:

  $$min\{ x^{t}(-Q_{objetivo})x\} = max\{ \sum_i p_ix_i\} = max\{p^{t}x\}$$

  > Observación: la dimension de la matriz es $n \times n$. Luego esto cambiará  

# Implementacion de la matriz de restriccion

Consideramos la restriccion: 

$w^{t}x \leq W$

Aqui el enfoque de optimizacion cambia. En vez de buscar soluciones que garanticen que se cumpla la restriccion, se buscaran soluciones que cumplan:

$(w^{t}x - W)^{2} \leq \epsilon$ 

Es decir, se intentara minimizar el error cometido al no cumplir la restriccion, y se agregará este término a la función objetivo.

Con esto en mente, queremos hallar una matriz $Q$ tal que:


$min \{Q_{peso}\}$ sea equivalente a que se respete $w^{t}x \leq W$.

## Necesidad de variables auxiliares
Para hallar esta matriz, debemos ampliar al vector $x$ ya que se necesitan agregar unas ciertas variables de slack $s_i$ para poder representar la igualdad. La cantidad de variables de slack a agregar es:

$$L =  \lfloor log_2(W)\rfloor+1$$

Entonces el vector $x$ queda:

$$x = [x_1 , x_2 , \dots, x_n, s_1, s_2,\dots,s_L ]$$

> Nuestra nueva matriz $Q_{objetivo}$ queda:

 \begin{pmatrix}
    Q_{objetivo} & 0 & \dots & 0 \\
    0 & 0_1 & \dots & 0 \\
    \vdots & \vdots & \ddots & \vdots \\
    0 & 0 & \dots & 0_L \\
  \end{pmatrix}

## Implementacion de la matriz de restriccion

Con esta aclaracion en mente, la matriz de restriccion de peso queda:

$$Q_{peso} = v^{t}*v - 2*W*D$$

Donde:

$$v = \left( w_1, w_2,\dots ,w_n, 2^0, 2^1,...,2^{L-1}, W+1 -2^{L-1} \right)$$

y $D$ es una matriz cuya diagonal principal es el vector $v$





In [ ]:
# Creamos instancia del problema
import numpy as np
import pandas as pd
import math

profits = np.array([18, 15, 10, 10, 18])
weights = np.array([19, 13, 10, 17, 10])
data = dict({'weight': weights,'profit': profits})
num_items = len(profits)
max_weight = int(np.floor(num_items / 2 * np.mean(weights)))
df = pd.DataFrame(data)
print("-------------------------------------")
print("Choose items from: \n ")
print(df)
print(f"with a max weight of: {max_weight}\n ")
print("-------------------------------------")

In [ ]:
# Calculamos la cantidad de slacks
num_slack = int(math.trunc(np.log2(max_weight))) + 1
print("Cantidad de slacks de peso: ", num_slack)

In [ ]:
def get_Q_objetivo(profits, num_slack):
  diagonal = np.r_[profits, np.zeros(num_slack)] # Creamos la diagonal de la matriz
  qubo = np.diag(diagonal) # Creamos la matriz diagonal
  return qubo

In [ ]:
def get_Q_Peso(max_weight, weights, num_slack):
 
    pesosSlacks = (2 ** (np.arange(num_slack - 1))).astype(float)
    pesosSlacks = np.r_[pesosSlacks, max_weight - sum(pesosSlacks)] # revisar esto.
    #pesosSlacks = np.r_[[1,2,4,8,16],[2]]
    vect = np.r_[weights, pesosSlacks]
    qubo = np.outer(vect, vect) - 2 * max_weight * np.diag(vect)  #No iria en el hamiltoniano

    return qubo

In [ ]:
# Formulacion del problema

q_objetivo = get_Q_objetivo(profits, num_slack)
print("Matriz q_objetivo: \n", q_objetivo)

Podemos notar que esta bien definida. Los profits en la diagonal principal seguido de $6$ ceros. 

In [ ]:
q_peso = get_Q_Peso(max_weight, weights, num_slack)
print("Matriz q_objetivo: \n", q_peso)

Ayuda con el analisis :) 

In [ ]:
# Armamos el modelo:

alpha = 2/9
qubo = -q_objetivo + alpha * q_peso

print("Dimensiones del problema: ", qubo.shape)



Para resolver el knapsack con $5$ items y peso maximo de $34$ necesitamos un total de $11$ qubits.

La formula general es:
Siendo $n$ la cantidad de items, $L$ la cantidad de slacks, se necesitaran un total de 
$N = n+L$ qubits para resolver el problema, o en funcion del peso maximo $W$:

$N = n+\lfloor log_2(W)\rfloor+1$ qubits.

En la siguiente seccion veremos como resolver el problema de optimizion utilizando dwave. 